In [1]:
!curl https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip -o /tmp/horse-or-human.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  142M  100  142M    0     0  8926k      0  0:00:16  0:00:16 --:--:-- 9850k


In [2]:
!curl https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -o /tmp/validation-horse-or-human.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.9M  100 10.9M    0     0  8213k      0  0:00:01  0:00:01 --:--:-- 8213k


In [3]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

In [4]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# Directory with our training horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# Directory with our training human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

In [5]:
train_horse_names = os.listdir(train_horse_dir)
train_human_names = os.listdir(train_human_dir)

validation_horse_hames = os.listdir(validation_horse_dir)
validation_human_names = os.listdir(validation_human_dir)

train_dir = '/tmp/horse-or-human/'
valid_dir = '/tmp/validation-horse-or-human/'

## Data Preprocessing using ImageDataGenerator:

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# rescale all images
train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)

# flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=128,
    class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## Build a Convolutional Neural Network:

In [10]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

In [13]:
def train_model():
    
    DESIRED_ACCURACY = 0.999
    
    # callback function
    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('accuracy') > DESIRED_ACCURACY):
                print("\nReached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True
                
    callbacks = myCallback()

    # building model
    # input image shape = 150, 150
    # binary classification
    model = tf.keras.models.Sequential([
        # first conv
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # second conv
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        # third conv
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        # flatten
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'),
        # output layer
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    # compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(lr=0.001),
                  metrics=['accuracy'])

    # train model
    history = model.fit(
        train_generator,
        steps_per_epoch=8,
        epochs=15,
        validation_data=valid_generator,
        validation_steps=8,
        callbacks=[callbacks])

In [14]:
train_model()

Epoch 1/15
8/8 [==============================] - 14s 2s/step - loss: 1.3105 - accuracy: 0.5017 - val_loss: 0.8784 - val_accuracy: 0.5000
Epoch 2/15
8/8 [==============================] - 14s 2s/step - loss: 0.6203 - accuracy: 0.6618 - val_loss: 0.3987 - val_accuracy: 0.8984
Epoch 3/15
8/8 [==============================] - 15s 2s/step - loss: 0.6804 - accuracy: 0.7742 - val_loss: 1.2814 - val_accuracy: 0.5117
Epoch 4/15
8/8 [==============================] - 13s 2s/step - loss: 0.8583 - accuracy: 0.7497 - val_loss: 0.7630 - val_accuracy: 0.7227
Epoch 5/15
8/8 [==============================] - 13s 2s/step - loss: 0.1836 - accuracy: 0.9488 - val_loss: 1.3109 - val_accuracy: 0.7500
Epoch 6/15
8/8 [==============================] - 13s 2s/step - loss: 0.1450 - accuracy: 0.9444 - val_loss: 1.0345 - val_accuracy: 0.8086
Epoch 7/15
8/8 [==============================] - 14s 2s/step - loss: 0.0915 - accuracy: 0.9689 - val_loss: 1.0862 - val_accuracy: 0.8203
Epoch 8/15
8/8 [==================

In [ ]:
# stop kernel
import os, signal
os.kill(os.getpid(), signal.SIGKILL)